In [8]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
df = pd.read_csv("/content/gdrive/MyDrive/tidydata.csv")
df['label'] = df['recommended'].apply(lambda x: 0 if x== True else 1)
X = df[['review']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(df.index.values, df.label.values, test_size=0.2, random_state=40, stratify=df.label.values)
df['type'] = ['tmp']*df.shape[0]
df.loc[X_train, 'type'] = 'train'
df.loc[X_test, 'type'] = 'test'
X_train_list = list(df[df.type=='train'].review.values)
Y_train_list = list(df[df.type=='train'].label.values)
tmp1 = []
tmp2 = []
for i in range(len(X_train_list)):
    if X_train_list[i]==X_train_list[i]:
        tmp1.append(X_train_list[i])
        tmp2.append(Y_train_list[i])
X_train_list = tmp1
Y_train_list = tmp2

X_test_list = list(df[df.type=='test'].review.values)
Y_test_list = list(df[df.type=='test'].label.values)
tmp1 = []
tmp2 = []
for i in range(len(X_test_list)):
    if X_test_list[i]==X_test_list[i]:
        tmp1.append(X_test_list[i])
        tmp2.append(Y_test_list[i])
X_test_list = tmp1
Y_test_list = tmp2

In [29]:
import json
with open("/content/gdrive/MyDrive/train_embeddings.json", "r") as file:
  train_emb = json.load(file)
with open("/content/gdrive/MyDrive/test_embeddings.json", "r") as file:
  test_emb = json.load(file)


In [23]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(train_emb, Y_train_list)
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
#NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(128, 2))
LR = LogisticRegression(random_state=0).fit(X_train, Y_train)
#NN.fit(X_train, Y_train)
LR.fit(X_train, Y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(random_state=0)

In [26]:
from sklearn.metrics import classification_report
y_pred = LR.predict(test_emb)
acc = 0
for i in range(len(y_pred)):
  if y_pred[i]==Y_test_list[i]:
    acc += 1
acc = acc/len(y_pred)
print(acc)
print(classification_report(y_pred, Y_test_list, digits=3))

0.8073333333333333
              precision    recall  f1-score   support

           0      0.807     0.652     0.721     11458
           1      0.808     0.904     0.853     18542

    accuracy                          0.807     30000
   macro avg      0.807     0.778     0.787     30000
weighted avg      0.807     0.807     0.802     30000



In [27]:
import numpy as np
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(Y_test_list, y_pred, pos_label=1)
metrics.auc(fpr, tpr)

0.8072001439296136

In [28]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test_list, y_pred)

0.8072001439296136

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X_train, Y_train = shuffle(train_emb, Y_train_list)
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train, Y_train)
#df_tfidf_test

ValueError: ignored

In [39]:
y=clf.predict(test_emb)
print(classification_report(y, Y_test_list, digits=3))

              precision    recall  f1-score   support

           0      0.738     0.776     0.756     19827
           1      0.786     0.749     0.767     21763

    accuracy                          0.762     41590
   macro avg      0.762     0.762     0.762     41590
weighted avg      0.763     0.762     0.762     41590



In [40]:
y_pred = clf.predict(test_emb)
import numpy as np
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(Y_test_list, y_pred, pos_label=1)
metrics.auc(fpr, tpr)

0.761799554080702

In [20]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, Y_train)

GaussianNB()

In [21]:
y_pred = gnb.predict(test_emb)
acc = 0
for i in range(len(y_pred)):
  if y_pred[i]==Y_test_list[i]:
    acc += 1
acc = acc/len(y_pred)
acc
print(classification_report(y_pred, Y_test_list, digits=3))

              precision    recall  f1-score   support

           0      0.755     0.587     0.661     26815
           1      0.466     0.655     0.545     14775

    accuracy                          0.611     41590
   macro avg      0.611     0.621     0.603     41590
weighted avg      0.653     0.611     0.620     41590



In [ ]:
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import torch.utils.data.dataset as Dataset
#train_x_ds = DataLoader(X_train_list, batch_size=32, num_workers=4)
#train_y_ds = DataLoader(Y_train_list, batch_size=32, num_workers=4)
X_train, Y_train = shuffle(train_emb, Y_train_list)
X_train = torch.tensor(train_emb)
Y_train = torch.tensor(Y_train_list)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
network = NeuralNetwork()
optimizer = optim.SGD(network.parameters(), lr=1e-5)
network.train()
train_losses = []
train_counter = []
epoch = 5
loss_fn = torch.nn.CrossEntropyLoss()
for e in range(epoch):
  output = network(X_train)
  loss = loss_fn(output, Y_train)
  train_losses.append(loss)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  pred = torch.max(F.softmax(output), 1)[1]
  pred = pred.data.numpy().squeeze()
  y = Y_train.data.numpy()
  acc = sum(pred==y)/len(y)
  print(acc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.5012134464281424
0.5012615037124252
0.5012494893913545
0.5011533748227888
0.5011713963043949


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X_train, Y_train = shuffle(train_emb, Y_train_list)
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train, Y_train)
#df_tfidf_test
print(classification_report(clf.predict(test_emb), Y_test_list))

              precision    recall  f1-score   support

           0       0.74      0.77      0.76     19891
           1       0.78      0.75      0.77     21699

    accuracy                           0.76     41590
   macro avg       0.76      0.76      0.76     41590
weighted avg       0.76      0.76      0.76     41590



In [ ]:
train_emb = pd.read_csv("/content/gdrive/MyDrive/df_train_w2v.csv")

In [ ]:
train_df = df[df.type=='train']
Y = []
for i in list(train_df['review'].index):
    if train_df['review'][i] != train_df['review'][i]:
      continue
    Y.append(train_df['label'][i])

In [ ]:
len(Y)

166468